# Ungraded Lab: Introduction to Keras callbacks

## Model methods that take callbacks

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

2024-08-04 13:20:47.595509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 13:20:47.595624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 13:20:47.737707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Version:  2.15.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteI32WFD/horses_or_humans-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteI32WFD/horses_or_humans-test.tfrecord*...:   0%…

Dataset horses_or_humans downloaded and prepared to ./tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


2024-08-04 13:21:10.114370: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43808: 7.10448, expected 6.11694
2024-08-04 13:21:10.114441: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43809: 7.52163, expected 6.53409
2024-08-04 13:21:10.114452: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43810: 7.97066, expected 6.98313
2024-08-04 13:21:10.114460: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43811: 7.81967, expected 6.83214
2024-08-04 13:21:10.114469: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43812: 7.66175, expected 6.67421
2024-08-04 13:21:10.114478: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43814: 8.19221, expected 7.20467
2024-08-04 13:21:10.114487: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 43815: 8.2081, expected 7.22056
2024-08-04 13:21:10.114495: 

 6/26 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4831 - loss: 0.7097

I0000 00:00:1722777672.617855     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22/26 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5581 - loss: 0.6769

2024-08-04 13:21:13.695278: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 7.01017, expected 6.07903
2024-08-04 13:21:13.695349: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 6.94217, expected 6.01103
2024-08-04 13:21:13.695378: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 8.2186, expected 7.28746
2024-08-04 13:21:13.695387: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 7.50591, expected 6.57477
2024-08-04 13:21:13.695395: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 6.63096, expected 5.69982
2024-08-04 13:21:13.695408: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 7.18353, expected 6.25239
2024-08-04 13:21:13.695417: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 10: 5.83342, expected 4.90228
2024-08-04 13:21:13.695424: E external/local_xla/xla/se

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.5703 - loss: 0.6695

2024-08-04 13:21:16.333205: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 28062: 4.13074, expected 3.58429
2024-08-04 13:21:16.333339: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 37997: 4.13074, expected 3.58429
2024-08-04 13:21:16.334194: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 178454: 4.22261, expected 3.6774
2024-08-04 13:21:16.334237: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 178913: 4.22897, expected 3.68377
2024-08-04 13:21:16.334316: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 188389: 4.22261, expected 3.6774
2024-08-04 13:21:16.334808: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 284752: 7.34198, expected 6.4183
2024-08-04 13:21:16.334837: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 284754: 7.61029, expected 6.68662
2024-08-04 13:21:16.33485

26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 170ms/step - accuracy: 0.5729 - loss: 0.6680 - val_accuracy: 0.4878 - val_loss: 0.6875
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6991 - loss: 0.5731 - val_accuracy: 0.8000 - val_loss: 0.4697
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7407 - loss: 0.5255 - val_accuracy: 0.8488 - val_loss: 0.3978
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8597 - loss: 0.4054 - val_accuracy: 0.8976 - val_loss: 0.3460
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9100 - loss: 0.3269 - val_accuracy: 0.9122 - val_loss: 0.2855
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8899 - loss: 0.3071 - val_accuracy: 0.9707 - val_loss: 0.2215
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9469 - loss: 0.2205 - val_accuracy: 0.8976 - val_loss: 0.2393
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9313 - loss: 0.2060 - val_accuracy: 0.9415 - val_loss: 0

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.keras', verbose=1),
          ])

Epoch 1/5

Epoch 1: saving model to weights.01-0.55.keras
26/26 - 3s - 123ms/step - accuracy: 0.6703 - loss: 0.6172 - val_accuracy: 0.7171 - val_loss: 0.5549
Epoch 2/5

Epoch 2: saving model to weights.02-0.52.keras
26/26 - 0s - 13ms/step - accuracy: 0.7798 - loss: 0.5154 - val_accuracy: 0.7024 - val_loss: 0.5169
Epoch 3/5

Epoch 3: saving model to weights.03-0.36.keras
26/26 - 0s - 13ms/step - accuracy: 0.8431 - loss: 0.4258 - val_accuracy: 0.9268 - val_loss: 0.3633
Epoch 4/5

Epoch 4: saving model to weights.04-0.31.keras
26/26 - 0s - 13ms/step - accuracy: 0.8382 - loss: 0.4019 - val_accuracy: 0.9317 - val_loss: 0.3141
Epoch 5/5

Epoch 5: saving model to weights.05-0.26.keras
26/26 - 0s - 13ms/step - accuracy: 0.8893 - loss: 0.3229 - val_accuracy: 0.9561 - val_loss: 0.2622


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('/kaggle/working/weights.04-0.31.keras', verbose=1)
          ])


Epoch 1: saving model to /kaggle/working/weights.04-0.31.keras
26/26 - 3s - 100ms/step - accuracy: 0.6557 - loss: 0.6274 - val_accuracy: 0.7659 - val_loss: 0.5710


In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.keras', verbose=1)
          ])

Epoch 1/2

Epoch 1: saving model to model.keras
26/26 - 2s - 96ms/step - accuracy: 0.6618 - loss: 0.6213 - val_accuracy: 0.8293 - val_loss: 0.5172
Epoch 2/2

Epoch 2: saving model to model.keras
26/26 - 0s - 13ms/step - accuracy: 0.7506 - loss: 0.5107 - val_accuracy: 0.7220 - val_loss: 0.4948


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 2s - 93ms/step - accuracy: 0.6119 - loss: 0.6528 - val_accuracy: 0.6244 - val_loss: 0.5916
Epoch 2/50
26/26 - 0s - 9ms/step - accuracy: 0.7336 - loss: 0.5086 - val_accuracy: 0.8439 - val_loss: 0.4239
Epoch 3/50
26/26 - 0s - 9ms/step - accuracy: 0.8017 - loss: 0.4395 - val_accuracy: 0.8634 - val_loss: 0.3880
Epoch 4/50
26/26 - 0s - 10ms/step - accuracy: 0.8418 - loss: 0.3657 - val_accuracy: 0.8976 - val_loss: 0.3121
Epoch 5/50
26/26 - 0s - 9ms/step - accuracy: 0.8625 - loss: 0.3270 - val_accuracy: 0.9024 - val_loss: 0.2800
Epoch 6/50
26/26 - 0s - 9ms/step - accuracy: 0.9088 - loss: 0.2806 - val_accuracy: 0.9268 - val_loss: 0.2341
Epoch 7/50
26/26 - 0s - 9ms/step - accuracy: 0.9282 - loss: 0.2427 - val_accuracy: 0.8098 - val_loss: 0.3902
Epoch 8/50
26/26 - 0s - 9ms/step - accuracy: 0.9404 - loss: 0.2063 - val_accuracy: 0.9854 - val_loss: 0.1545
Epoch 9/50
26/26 - 0s - 9ms/step - accuracy: 0.9526 - loss: 0.1760 - val_accuracy: 0.9805 - val_loss: 0.1273
Epoch 10/50
26/26

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [19]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.5807 - loss: 0.6618 - val_accuracy: 0.6488 - val_loss: 0.6281
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7031 - loss: 0.6047 - val_accuracy: 0.7073 - val_loss: 0.5829
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7587 - loss: 0.5562 - val_accuracy: 0.8537 - val_loss: 0.5146
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7799 - loss: 0.5173 - val_accuracy: 0.8976 - val_loss: 0.4705
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8154 - loss: 0.4850 - val_accuracy: 0.9073 - val_loss: 0.4346


In [20]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.627737,0.647765,0.648780,0.628051
1,1,0.710462,0.597093,0.707317,0.582894
2,2,0.762774,0.549425,0.853659,0.514637
3,3,0.779805,0.520654,0.897561,0.470471
4,4,0.801703,0.483262,0.907317,0.434639


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.5603 - loss: 0.6678 - val_accuracy: 0.6976 - val_loss: 0.6195 - learning_rate: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7488 - loss: 0.5949 - val_accuracy: 0.8439 - val_loss: 0.5863 - learning_rate: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8158 - loss: 0.5661 - val_accuracy: 0.9024 - val_loss: 0.5669 - learning_rate: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8614 - loss: 0.5606 - val_accuracy: 0.8000 - val_loss: 0.5690 - learning_rate: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8299 - loss: 0.5517 - val_accuracy: 0.8439 - val_loss: 0.5634 - learning_rat

In [22]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.5375 - loss: 0.7123 - val_accuracy: 0.8000 - val_loss: 0.5737 - learning_rate: 0.0100
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8098 - loss: 0.5370 - val_accuracy: 0.8927 - val_loss: 0.4636 - learning_rate: 0.0100
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8047 - loss: 0.4873 - val_accuracy: 0.9073 - val_loss: 0.4304 - learning_rate: 0.0100
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8857 - loss: 0.4033 - val_accuracy: 0.8732 - val_loss: 0.3545 - learning_rate: 0.0100
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8789 - loss: 0.3544 - val_accuracy: 0.9317 - val_loss: 0.2932 - learning_rate: 0.0100
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8945 - loss: 0.2995 - val_accuracy: 0.9561 - val_loss: 0.2432 - learning_rate: 0.0100
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9325 - loss: 0.2462 - val_accuracy: 0.94

In [24]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 1184), started 0:00:17 ago. (Use '!kill 1184' to kill it.)

In [25]:
!kill 1184

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
